nouvelles specs on part d'un nouvel atlas de 10 régions et on voit quels sont les voxels qui sont en correlations avec ces régions  <br> 
on va d'abord determiner combien il y de regions, puis on va les calculer à partir de l'atlas utilisateur

In [1]:
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine

from nipype.interfaces.utility import Function, IdentityInterface

from nipype.interfaces.fsl.maths import MathsCommand
from nipype.interfaces.fsl.utils import PlotMotionParams   # to plot moco variabl
from nipype import SelectFiles, Node, MapNode

In [2]:
# creation of a subworflow to process ROI Correlations
connectivity = pe.Workflow(name='connectivity')

In [3]:
arimadir = '/scratch/user/hirsch/datadir4/data_results_py/functionnal/arima/arimaResidus'
atlasdir = '/scratch/user/hirsch/datadir/data_set/t0009/repos01/Atlases'


from nipype import SelectFiles, Node
templates = dict(arimaFile=arimadir+ "/" + "*.nii.gz",
                 atlasFile=atlasdir + "/" + "atlas*.nii")

filesource = Node(SelectFiles(templates), "filesource")
filesource.inputs.subject_id = "subj1"
filesource.outputs.get()

{'arimaFile': <undefined>, 'atlasFile': <undefined>}

In [4]:
# lets first determine how many regions have been defined by the user

def identifyRegionNb(atlas_file):
    """compute Regions of Interest number, their integer values from atlas file given by the user, 
    output is the array of int number, each one representing a different ROI """
    
    import numpy as np     
    import nibabel as nib     
    import os 
    
    atlas_img=nib.load(atlas_file)         
    atlas_array=np.asarray(atlas_img.dataobj).copy() # Avoid caching the proxy image
    
    atlas_array = atlas_array[atlas_array>0]
    # np.unique(a)
    regions = np.unique(atlas_array)
    print regions
    
    regionsNb = len(regions) 
    
    return regions
    

In [5]:
# lets select region i
# input is atlasFile, output is maskFile with region 1

def selectRegion(atlas_file, n):
    
    import numpy as np     
    import nibabel as nib     
    import os 
    
    atlas_img=nib.load(atlas_file)         
    atlas_array=np.asarray(atlas_img.dataobj).copy() # Avoid caching the proxy image
    
    # test n = 1
    region_array = atlas_array
    # binary mask the resulting image
    region_array[atlas_array <> n] = 0
    region_array[atlas_array == n] = 1
    
    region_image = nib.Nifti1Image(region_array, atlas_img.affine, atlas_img.header)
    
    result = os.getcwd() + '/' + 'region_image' + str(n) + '.nii'
    nib.save(region_image, result)
    
    return result


In [6]:
def identifyRegionAtlases(atlas_file):
    """compute Regions of Interest number, their integer values from atlas file given by the user,
    then compute the atlas image for each of n values, then
    output is the array of regions atlases, each one representing a different ROI """
    
    roi_img_list =[]
    roi_nbs = identifyRegionNb(atlas_file)
    for i in roi_nbs:
        
        roi_img_list.append(selectRegion(atlas_file, i))
        
    return roi_img_list
        
    

In [7]:
# test fct 
identifyRegionAtlases('/scratch/user/hirsch/datadir/data_set/t0009/repos01/Atlases/atlas_2reg.nii')

[1 2]


['/homes_unix/hirsch/_ipnotebooks/region_image1.nii',
 '/homes_unix/hirsch/_ipnotebooks/region_image2.nii']

In [8]:
# node to compute a mask of region 1 of user atlas
IdentifyRegions = Node(Function(input_names=['atlas_file'],
                                output_names=['regions_images_list'],
                                function=identifyRegionAtlases),
                                name='IdentifyRegions')



connectivity.connect(filesource, "atlasFile", IdentifyRegions, "atlas_file")


In [9]:
# here comes the map node, we have to iterate the treatment for each region identified by previous step
# b = pe.MapNode(interface=B(), name="b", iterfield=['in_file']) 
# http://nipype.readthedocs.io/en/latest/users/mapnode_and_iterables.html

# lets calculate the mean residu signal eg. time courses in every region 


from nipype.interfaces.fsl.utils import ImageMeants

regMeants = MapNode(ImageMeants(), name="regMeants", iterfield=['mask'])     
regMeants.inputs.ignore_exception = False     
regMeants.inputs.order = 1     
regMeants.inputs.output_type = 'NIFTI_GZ'     
regMeants.inputs.terminal_output = 'stream'     
connectivity.connect(filesource, "arimaFile" , regMeants, "in_file")   
connectivity.connect(IdentifyRegions, "regions_images_list", regMeants, "mask")


reg1 = selectRegion('/scratch/user/hirsch/datadir/data_set/t0009/repos01/Atlases/atlas_2reg.nii', 2)
print reg1

In [10]:
# data sink
datasink = pe.Node(nio.DataSink(), name='datasink')
datasink.inputs.base_directory = '/scratch/user/hirsch/datadir4/data_results_py'
connectivity.connect(regMeants,  'out_file', datasink, 'functionnal.regMeants')

In [11]:
connectivity.run()

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node filesource in dir: /tmp/tmpb_xDCM/connectivity/filesource
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node IdentifyRegions in dir: /tmp/tmpxiBaMO/connectivity/IdentifyRegions
ERROR:workflow:['Node IdentifyRegions failed to run on host pipeau.dgin.bor.']
INFO:workflow:Saving crash info to /homes_unix/hirsch/_ipnotebooks/crash-20170511-163954-hirsch-IdentifyRegions-b163b35d-2911-4f33-bfcc-5525ab7b3eac.pklz
INFO:workflow:Traceback (most recent call last):
  File "/srv/shares/softs/anaconda2/lib/python2.7/site-packages/nipype/pipeline/plugins/linear.py", line 39, in run
    node.run(updatehash=updatehash)
  File "/srv/shares/softs/anaconda2/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line 394, in run
    self._run_interface()
  File "/srv/shares/softs/anaconda2/lib/python2.7/site-packages/nipype/pipeline/engine/nodes.py", line

RuntimeError: Workflow did not execute cleanly. Check log for details

# fin du pipe